<a href="https://colab.research.google.com/github/laraselinseyahi/Diabetic-Retinopathy-Classification-using-Deep-Learning/blob/main/CS230_MultiClassClassifier_LaraSelinSeyahi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multiclass Classification with ResNet-50 for Diabetic Retinopathy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/


/content/drive/MyDrive
'!#1Literacy narrative_Lara seyahi.gdoc'
'#1Literacy narrative_Lara seyahi.gdoc'
 20220628_LS.gslides
'2022 B-SURP Interenships.gdoc'
 21-22units.gdoc
 22units.gdoc
'2Copy of Psych80_Essay#2.gdoc'
'#2Op-Ed_Lara Seyahi.gdoc'
'#3AnnotatedBib_LaraSeyahi.gdoc'
'3 - minutes(average).py'
 70b-40-40-prospect_theory.json
 abstract.gdoc
'accenture info chat.gdoc'
'A comparison of the ideal and actual affect of college students in Turkey and US.gdoc'
'Adobe Scan Feb 1, 2022.pdf'
'Adobe Scan Jan 19, 2022.pdf'
'ANA COPY of daily 3 - hatay.gdoc'
'ancova_final!.csv'
'Annotated bibliography.gdoc'
'Annotated Bibliography .gdoc'
'annotated bib pwr 2.gdoc'
'annotated bib_pwr 2.gdoc'
'annotation tips.gdoc'
'ApprovalLetter - 66825 (1).pdf'
'aras essayler.gdoc'
'aras odev (1).gdoc'
'aras odev (2).gdoc'
'aras odev.gdoc'
'aras odevi.gdoc'
'Aras Türkçe Projesi.gslides'
'artik hayatimizda olmayan isimler.gdoc'
 ask.gdoc
'assignment 5.gdoc'
'A summer Day in De Young.gdoc'
'avi graph .gs

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = 'diabetic_retinopathy'

# Class definitions
classes = ["healthy", "mild_dr", "moderate_dr", "proliferate_dr", "severe_dr"]
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
# creates the following dict {"healthy": 0, "mild_dr": 1, "moderate_dr": 2, "proliferate_dr": 3, "severe_dr": 4}



In [ ]:
# Collect file paths and labels
file_paths = []
labels = []

# TensorFlow works efficiently with datasets when file paths and labels are paired,
# as it can process the data in batches and apply preprocessing.
# This ensures that each image is correctly paired with its label, which is crucial for supervised learning.
# This part collects all the image file paths and their corresponding labels.
for cls in classes:
    class_dir = os.path.join(data_dir, cls)
    for file in os.listdir(class_dir):
        file_paths.append(os.path.join(class_dir, file))
        labels.append(class_to_idx[cls])

# Split the dataset into train, validation, and test sets (70% train, remaining eval&test)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    file_paths, labels, test_size=0.3, stratify=labels, random_state=42
)
# 20% eval, 10% test
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.33, stratify=temp_labels, random_state=42
)

# Sanity check
print(f"Train: {len(train_paths)}, Validation: {len(val_paths)}, Test: {len(test_paths)}")

# Define image size and batch size
IMG_SIZE = (224, 224) # ResNet-50 input size
BATCH_SIZE = 32

# Function to preprocess images
def preprocess_image(file_path, label):
    # Load the image
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    # Perform data augmentation for labels not in [0, 2]
    if label == 1 or label == 3 or label == 4:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
        image = tf.image.random_crop(image, size=(int(IMG_SIZE[0] * 0.9), int(IMG_SIZE[1] * 0.9), 3))
        image = tf.image.resize(image, IMG_SIZE)  # Resize back to original size after cropping
    image = tf.keras.applications.resnet50.preprocess_input(image) #ResNet-50's requirement of [-1, 1]
    return image, label

# Create a TensorFlow dataset
def create_dataset(file_paths, labels, shuffle=True):
    # pair file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    # map function applies preprocess_image function to each image
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(file_paths))
    dataset = dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

# Create train, validation, and test datasets
train_dataset = create_dataset(train_paths, train_labels)
val_dataset = create_dataset(val_paths, val_labels, shuffle=False)
test_dataset = create_dataset(test_paths, test_labels, shuffle=False)

# Sanity check over the first batch
# for images, labels in train_dataset.take(1):
#    print(f"Batch shape: {images.shape}, Labels: {labels.numpy()}")


Train: 1974, Validation: 566, Test: 280


In [ ]:
# RESNET Experimenting

# RESNET

base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

for layer in base_model.layers:
  layer.trainable = False

# Add a new top layer for our 5-class classification
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(), # pooling to reduce output size
    tf.keras.layers.Dense(5, activation='softmax')  # 5 classes

    # adding more layer increases the ability of your model, but the model is already good
])

base_learning_rate = 0.001
# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,  # Adjust based on your needs
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        # Early stopping for stopping training when validation performance stops improving
        tf.keras.callbacks.ModelCheckpoint('resnet50_best_model.keras', save_best_only=True)
        # saves best performing model during training
    ]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.2f}")


# 1) explore more data augmentation methods
  # - does changing methods help? removing help?
  # - try removing data from healthy class

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │          10,245 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,597,957 (90.02 MB)

 Trainable params: 10,245 (40.02 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 185s 3s/step - accuracy: 0.6307 - loss: 1.0342 - val_accuracy: 0.8127 - val_loss: 0.5459
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 23s 183ms/step - accuracy: 0.8168 - loss: 0.5181 - val_accuracy: 0.8445 - val_loss: 0.4259
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 39s 169ms/step - accuracy: 0.8535 - loss: 0.4184 - val_accuracy: 0.8304 - val_loss: 0.4515
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 23s 177ms/step - accuracy: 0.8706 - loss: 0.3539 - val_accuracy: 0.8675 - val_loss: 0.3705
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 43s 185ms/step - accuracy: 0.8795 - loss: 0.3262 - val_accuracy: 0.8710 - val_loss: 0.3659
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 23s 168ms/step - accuracy: 0.8686 - loss: 0.3372 - val_accuracy: 0.8569 - val_loss: 0.3826
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 41s 186ms/step - accuracy: 0.8785 - loss: 0.3129 - val_accuracy: 0.8746 - val_loss: 0.3413
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 40s 158ms/step - accuracy: 0.8877 - loss: 0.2933 - val_accura

In [ ]:
# RESNET Experimenting

# RESNET

base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

for layer in base_model.layers[-10:]:
  layer.trainable = True

# Add a new top layer for our 5-class classification
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(), # pooling to reduce output size
    tf.keras.layers.Dense(5, activation='softmax')  # 5 classes

    # adding more layer increases the ability of your model, but the model is already good
])

base_learning_rate = 0.001
# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,  # Adjust based on your needs
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint('resnet50_best_model.keras', save_best_only=True)
    ]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.2f}")


# 1) explore more data augmentation methods
  # - does changing methods help? removing help?
  # - try removing data from healthy class

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │          10,245 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,597,957 (90.02 MB)

 Trainable params: 23,544,837 (89.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1058s 4s/step - accuracy: 0.6778 - loss: 0.9600 - val_accuracy: 0.3074 - val_loss: 454.9324
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 62s 429ms/step - accuracy: 0.7755 - loss: 0.5218 - val_accuracy: 0.6201 - val_loss: 11.8420
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 39s 390ms/step - accuracy: 0.8356 - loss: 0.3943 - val_accuracy: 0.6307 - val_loss: 2.5769
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 41s 381ms/step - accuracy: 0.8455 - loss: 0.3354 - val_accuracy: 0.5106 - val_loss: 6.9450
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 41s 391ms/step - accuracy: 0.8645 - loss: 0.3054 - val_accuracy: 0.8092 - val_loss: 0.5536
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 42s 379ms/step - accuracy: 0.8690 - loss: 0.2814 - val_accuracy: 0.6802 - val_loss: 3.8888
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 40s 357ms/step - accuracy: 0.8700 - loss: 0.2934 - val_accuracy: 0.6025 - val_loss: 12.5622
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 40s 355ms/step - accuracy: 0.8728 - loss: 0.2864 - val_a

In [ ]:
# RESNET

base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

for layer in resnet_model.layers:
  layer.trainable = False

# Add a new top layer for our 5-class classification
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(), # pooling to reduce output size
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5), # regularization
    tf.keras.layers.Dense(5, activation='softmax')  # 5 classes
])

base_learning_rate = 0.001
# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,  # Adjust based on your needs
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
        # Early stopping for stopping training when validation performance stops improving
        tf.keras.callbacks.ModelCheckpoint('resnet50_best_model.keras', save_best_only=True)
        # saves best performing model during training
    ]
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.2f}")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,113,541 (91.99 MB)

 Trainable params: 24,060,421 (91.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 352s 4s/step - accuracy: 0.5329 - loss: 1.4933 - val_accuracy: 0.3569 - val_loss: 17746.8164
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 42s 438ms/step - accuracy: 0.6034 - loss: 1.0562 - val_accuracy: 0.3551 - val_loss: 7088.2329
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 41s 436ms/step - accuracy: 0.6314 - loss: 1.0219 - val_accuracy: 0.4700 - val_loss: 6.0775
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 42s 437ms/step - accuracy: 0.6283 - loss: 1.0167 - val_accuracy: 0.6484 - val_loss: 1.1320
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 39s 364ms/step - accuracy: 0.6433 - loss: 0.9368 - val_accuracy: 0.5724 - val_loss: 1.6182
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 41s 431ms/step - accuracy: 0.6357 - loss: 0.9380 - val_accuracy: 0.6590 - val_loss: 0.9240
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 42s 435ms/step - accuracy: 0.6663 - loss: 0.9029 - val_accuracy: 0.6519 - val_loss: 0.9153
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 37s 401ms/step - accuracy: 0.6528 - loss: 0.8832 - val